In [ ]:
data = pd.read_csv('/content/sentimentdataset.csv')

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding, SpatialDropout1D, Concatenate, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Convert 'Sentiment' column to numerical labels
sentiment_encoder = LabelEncoder()
data['Sentiment'] = sentiment_encoder.fit_transform(data['Sentiment'])

# Extract features for the temporal-based approach
data['Hour_sin'] = np.sin(2 * np.pi * data['Hour'] / 24)
data['Hour_cos'] = np.cos(2 * np.pi * data['Hour'] / 24)

# Split the data into training and testing sets
X = data[['Text', 'Sentiment', 'Hour_sin', 'Hour_cos']]
y = data['Sentiment']  # Assuming 'Mood' is the target variable
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Text-based Approach
tokenizer = Tokenizer(num_words=1000)
tokenizer.fit_on_texts(X_train['Text'])
X_text_train = tokenizer.texts_to_sequences(X_train['Text'])
X_text_test = tokenizer.texts_to_sequences(X_test['Text'])
X_text_train = pad_sequences(X_text_train, maxlen=100)
X_text_test = pad_sequences(X_text_test, maxlen=100)

# Sentiment-based Approach
X_sentiment_train = X_train['Sentiment'].values.reshape(-1, 1)
X_sentiment_test = X_test['Sentiment'].values.reshape(-1, 1)

# Temporal-based Approach
X_temporal_train = X_train[['Hour_sin', 'Hour_cos']].values
X_temporal_test = X_test[['Hour_sin', 'Hour_cos']].values

num_classes = len(np.unique(y))

# Define the RNN model
text_input = Input(shape=(X_text_train.shape[1],))
embedded_text = Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=64, input_length=X_text_train.shape[1])(text_input)
text_lstm = LSTM(64)(embedded_text)

sentiment_input = Input(shape=(1,))
sentiment_dense = Dense(32, activation='relu')(sentiment_input)

temporal_input = Input(shape=(2,))
temporal_dense = Dense(32, activation='relu')(temporal_input)

concatenated = Concatenate()([text_lstm, sentiment_dense, temporal_dense])
output = Dense(num_classes, activation='softmax')(concatenated)

model = Model(inputs=[text_input, sentiment_input, temporal_input], outputs=output)

# Compile the model
model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit([X_text_train, X_sentiment_train, X_temporal_train], y_train, epochs=20, batch_size=32, validation_split=0.2)




Epoch 1/20
15/15 [==============================] - 7s 102ms/step - loss: 20.5230 - accuracy: 0.0171 - val_loss: 18.0568 - val_accuracy: 0.0684
Epoch 2/20
15/15 [==============================] - 1s 58ms/step - loss: 12.0976 - accuracy: 0.0577 - val_loss: 15.3358 - val_accuracy: 0.0000e+00
Epoch 3/20
15/15 [==============================] - 1s 61ms/step - loss: 8.4582 - accuracy: 0.0321 - val_loss: 13.7273 - val_accuracy: 0.0684
Epoch 4/20
15/15 [==============================] - 1s 59ms/step - loss: 6.6876 - accuracy: 0.0983 - val_loss: 13.1608 - val_accuracy: 0.0769
Epoch 5/20
15/15 [==============================] - 1s 59ms/step - loss: 5.8536 - accuracy: 0.1090 - val_loss: 12.9577 - val_accuracy: 0.0000e+00
Epoch 6/20
15/15 [==============================] - 1s 59ms/step - loss: 5.3489 - accuracy: 0.1239 - val_loss: 12.6251 - val_accuracy: 0.1197
Epoch 7/20
15/15 [==============================] - 1s 60ms/step - loss: 5.0130 - accuracy: 0.1667 - val_loss: 12.4767 - val_accuracy: 0.

In [ ]:
# Predict labels for the testing data
y_pred = model.predict([X_text_test, X_sentiment_test, X_temporal_test])
predicted_labels = np.argmax(y_pred, axis=1)

# Create a DataFrame with the text and predicted labels
output_df = pd.DataFrame({'Text': X_test['Text'], 'Predicted_Mood': predicted_labels})

# Save the DataFrame to a CSV file
output_df.to_csv('sulakpredicted_labels.csv', index=False)

5/5 [==============================] - 0s 17ms/step


In [ ]:
# Predict labels for the testing data
y_pred = model.predict([X_text_test, X_sentiment_test, X_temporal_test])
predicted_labels = np.argmax(y_pred, axis=1)

# Create a DataFrame with the text and predicted labels
output_df = pd.DataFrame({'Text': X_test['Text'], 'Predicted_Mood': predicted_labels})

# Save the DataFrame to a CSV file
output_df.to_csv('sulpredicted_labels.csv', index=False)


5/5 [==============================] - 1s 14ms/step


In [ ]:
pred1 = pd.read_csv('/content/sulpredicted_labels.csv')

,Text,Predicted_Mood
0,Exploring the world of digital art. It's never...,110
1,Feeling inspired after attending a workshop. ...,214
2,"Eyes wide open in the night, fearful shadows ...",152
3,"A soul weathered by the storm of heartbreak, s...",172
4,"Attended a wine tasting event, savoring the ri...",172


In [ ]:
moodtest_data = pd.read_csv("/content/moodtest.csv")


In [ ]:
actual_values = moodtest_data['Sentiment']
predicted_values = pred['Predicted_labels']

In [ ]:
report = classification_report(actual_values, predicted_values)

# Print the classification report
print("Classification Report:")
print(report)

Classification Report:
               precision    recall  f1-score   support

   Excitement       0.00      0.00      0.00         0
         Bad        1.00      1.00      1.00         6
 Contentment        1.00      1.00      1.00         8
   Curiosity        1.00      1.00      1.00         4
 Embarrassed        1.00      1.00      1.00         8
   Excitement       0.00      0.00      0.00         0
  Excitement        0.95      1.00      0.98        20
   Gratitude        1.00      1.00      1.00         5
       Happy        0.92      0.79      0.85        14
        Hate        1.00      1.00      1.00         6
         Joy        0.94      0.97      0.95        30
        Joy         0.00      0.00      0.00         0
 Mischievous        1.00      1.00      1.00         2
     Neutral        1.00      0.79      0.88        14
    Positive        1.00      1.00      1.00         1
      Relief        1.00      1.00      1.00         1
         Sad        1.00      1.00      1

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
